In [6]:
from simulator.reporters import TransitionReporter
from simulator.core import Task

In [3]:
transitionList = TransitionReporter.getListFromPickle("reports/reportmobile0xa089b.pkl")

In [8]:
task: Task = transitionList[0].task
task.arrivalTime()

0.036496461390015035